In [4]:
import pandas as pd
from openai import OpenAI
from pinecone import Pinecone
from keys.keys import OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME
from functions.query_index import query_index
from functions.read_docs import read_docs
from functions.generate_answer import generate_answer
# from functions.query import query

In [5]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = 'text-embedding-ada-002'

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

In [6]:
file_name = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset.xlsx" 
sheet = "QnA"  
df = pd.DataFrame(data=pd.read_excel(io=file_name, sheet_name=sheet), columns=["Type", "Question", "Answer"])

df.head(5)

,Type,Question,Answer
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 10:00 AM to 10:00 PM.
1,General Information,Is there free Wi-Fi available in Siam Paragon?,Free Wi-Fi available throughout the area of Si...
2,General Information,Are pets allowed in Siam Paragon?,"No, pets are not allowed in Siam Paragon."
3,General Information,What is the contact number for Siam Paragon's ...,Siam Paragon can be contacted at 02-610-8000 a...
4,General Information,Are there any prayer rooms available?,"Yes, there is a Muslim prayer room available o..."


In [10]:
test = df.head(5)

In [15]:
def query(row, client, model, index):
    prompt = row['Question']
    
    var_fn = {}
    
    index_result = query_index(client=client, model=model, index=index, prompt=prompt)
    var_fn.update({'index_result': index_result})
    
    file_ids = []
    for res in index_result:
        file_id = res.split(': ')[1]
        file_ids.append(file_id)
        
    contents = read_docs(file_ids=file_ids)
    
    response = generate_answer(client=client, prompt=prompt, contents=contents)
    var_fn.update({'response': response})
    
    return var_fn

In [17]:
test['GPT'] = test.apply(query, args=(client, model, index), axis = 1)[0]['response']

KeyError: 0

In [13]:
test

,Type,Question,Answer,GPT
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 10:00 AM to 10:00 PM.,{'index_result': ['Bangkok Bank: 1NUsklmv3k0Qm...
1,General Information,Is there free Wi-Fi available in Siam Paragon?,Free Wi-Fi available throughout the area of Si...,{'index_result': ['Bangkok Bank: 1NUsklmv3k0Qm...
2,General Information,Are pets allowed in Siam Paragon?,"No, pets are not allowed in Siam Paragon.",{'index_result': ['DS: 1A8Tw_dD9tRQZt9fy328nMd...
3,General Information,What is the contact number for Siam Paragon's ...,Siam Paragon can be contacted at 02-610-8000 a...,{'index_result': ['Bangkok Bank: 1NUsklmv3k0Qm...
4,General Information,Are there any prayer rooms available?,"Yes, there is a Muslim prayer room available o...",{'index_result': ['restroom: 13FAAOime7mFWQf-o...


In [14]:
test['GPT'][0]['response']

'\n\nAnswer: The Siam Paragon branch of Bangkok Bank is open daily from 10:30am to 8:00pm.'